# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

## My Imports:

In [10]:
from sklearn.metrics import make_scorer

Data Observation:

In [11]:
df_train["gender"].value_counts()

m    575
f    178
Name: gender, dtype: int64

------------
## Helper Functions

### Handling the target column

In [12]:
def Preparing_target_column(df):
    df["is_male"] = (df["gender"] == 'm').astype(int)
    df.drop("gender", axis=1, inplace=True)
    
def back_to_gender_column(df):
    df["gender"] = df["is_male"].copy().replace(1, 'm').replace(0, 'f')
    df.drop("is_male", axis=1, inplace=True)

### Model revaluation

In [13]:
def macro_average_f1(clf, X, y):
    return round(cross_val_score(clf, X, y, scoring=make_scorer(f1_score, average='macro'), cv=10).mean(), 3)

### To get the df with the predicted as a column

In [14]:
def get_df_predicted(df, pred):
    pred = pd.Series(pred)
    df_copy = df.copy()
    df_copy['is_male'] = pred
    back_to_gender_column(df_copy)
    return df_copy

### To display the results of the prediction
5 top rows and 5 bottom rows

In [15]:
def top_and_bottom_df(df, pred, num=5):
    df_pred = get_df_predicted(df, pred)
    df_cat = pd.concat([df_pred[: num].copy(), df_pred[-num:].copy()])
    return df_cat

------------
## Initial Experience With The Data

In [16]:
Preparing_target_column(df_train)
df_train.head()

,story,is_male
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",1
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,1
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,0
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",1
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,0


## Defines X and y

In [17]:
X_train = df_train["story"]
y_train = df_train["is_male"]
X_test = df_test["story"]

In [18]:
text_clf_nb = Pipeline([
    ('vect', TfidfVectorizer(min_df=2)),
    ('norm', preprocessing.Normalizer(norm='l2')),
    ('clf_nb', MultinomialNB()),
])

macro_average_f1(text_clf_nb, X_train, y_train)

0.433

In [19]:
pipe = text_clf_nb.fit(X_train, y_train)
y_pred = text_clf_nb.predict(X_test)

top_and_bottom_df(df_test, y_pred)

,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",m
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,m
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",m
322,322,היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...,m


------------
## Improvement Attempts

In [20]:
def grid_search_pipeline(mods_with_params, norm_params, vecs_with_params, X, y):
    print ("Grid Search Pipeline:\n------------------\n")
    
    best_clf = None
    best_score = 0
    for mod, mod_params in mods_with_params.items():
        for vect, vect_params in vecs_with_params.items():
            text_clf_nb = Pipeline([
            ('vect', vect),
            ('norm', preprocessing.Normalizer()),
            ('clf', mod),
            ])
            
            print (f" mod: {mod}\n vect: {vect}\n\n")
            params = mod_params.copy()
            params.update(vect_params)
            params.update(norm_params)
            
            search_clf = GridSearchCV(text_clf_nb, params, scoring=make_scorer(f1_score, average='macro'), cv=10)
            search_clf.fit(X, y)
            
            print("Best parameter set is:",search_clf.best_params_,"\n\n And its score was:", round(search_clf.best_score_, 3))
            if(best_score < search_clf.best_score_):
                best_score = search_clf.best_score_
                best_clf = search_clf
            print("------------------\n")
    return best_clf
    
    

In [21]:
X_train = df_train["story"]
y_train = df_train["is_male"]
X_test = df_test["story"]

### General check of models with different parameters:

In [22]:
models_with_params = {
    MultinomialNB(): dict(),
    KNeighborsClassifier(): {'clf__n_neighbors': [3, 5, 7]},
    DecisionTreeClassifier(): {'clf__max_depth': [3, 5, 7], "clf__min_samples_split": [5, 10, 20]},
}

normalizer_params = {'norm__norm': ['l2', 'l1']}

vectorizers_with_params = {
    TfidfVectorizer(): {
        'vect__ngram_range': [(1,1)], 'vect__min_df': [1, 2, 0.1],
        'vect__max_df': [1., 0.9], 'vect__sublinear_tf': [True, False],
    },
    CountVectorizer(): {
        'vect__ngram_range': [(1,1)], 'vect__min_df': [1, 2, 0.1],
        'vect__max_df': [1., 0.9],
    },
}

In [23]:
best_clf = grid_search_pipeline(models_with_params, normalizer_params, vectorizers_with_params, X_train, y_train)

Grid Search Pipeline:
------------------

 mod: MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
 vect: TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


Best parameter set is: {'norm__norm': 'l2', 'vect__max_df': 1.0, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__sublinear_tf': True} 

 And its score was: 0.433
------------------

 mod: MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
 vect: CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf

In [24]:
print("Best Estimator:\n", best_clf.best_estimator_)
print("Best Parameters:\n", best_clf.best_params_)

print("\nAverage f1: ", round(best_clf.best_score_, 3))

Best Estimator:
 Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=True,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('norm', Normalizer(copy=True, norm='l2')),
                ('clf',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
  

----------------------
### KNN gave us the best result with Tfidf

'n_neighbors': 3, 'norm': 'l2', 'max_df': 1.0,

'min_df': 1, 'ngram_range': (1, 1), 'sublinear_tf': True}

### And also DecisionTree with CountVectorizer:

'max_depth': 7, 'min_samples_split': 5,
'norm': 'l2',

'max_df': 0.9, 'min_df': 2, 'ngram_range': (1, 1)}

-------------

## Trying more parameters on the most successful combinations

### KNN gave us the best result with Tfidf

In [25]:
models_with_params = {
    KNeighborsClassifier(): {'clf__n_neighbors': [3, 9, 11], 'clf__metric': ['euclidean', 'manhattan'],
                             'clf__weights': ['uniform', 'distance']},
}

normalizer_params = {'norm__norm': ['l2']}

vectorizers_with_params = {
    TfidfVectorizer(): {
        'vect__ngram_range': [(1,1), (2,2), (1,2)], 'vect__min_df': [1],
        'vect__max_df': [1., 0.8], 'vect__sublinear_tf': [True, False],
    },
}

best_clf_knn = grid_search_pipeline(models_with_params, normalizer_params, vectorizers_with_params, X_train, y_train)

Grid Search Pipeline:
------------------

 mod: KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
 vect: TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


Best parameter set is: {'clf__metric': 'euclidean', 'clf__n_neighbors': 3, 'clf__weights': 'distance', 'norm__norm': 'l2', 'vect__max_df': 1.0, 'vect__min_df': 1, 'vect__ngram_range': (1, 2), 'vect__sublinear_tf': True} 

 And its score was: 0.641
-------------

### Evaluation And Prediction:

In [26]:
text_clf_knn = best_clf_knn.best_estimator_
average_f1 = round(best_clf_knn.best_score_, 3)

pipe_knn = text_clf_knn.fit(X_train, y_train)
y_pred = text_clf_knn.predict(X_test)

print('Average f1: ', average_f1)
top_and_bottom_df(df_test, y_pred)

Average f1:  0.641


,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",f
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",m
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,m
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",m
322,322,היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...,m


### DecisionTree with CountVectorizer:

In [27]:
models_with_params = {
    DecisionTreeClassifier(): {'clf__max_depth': [7, 9, 11], "clf__min_samples_split": [20, 30],
                              'clf__criterion': ['gini', 'entropy', 'log_loss']},
}

normalizer_params = {'norm__norm': ['l2']}

vectorizers_with_params = {
    CountVectorizer(): {
        'vect__ngram_range': [(1,1), (1,2), (2,2)], 'vect__min_df': [1],
        'vect__max_df': [1., 0.9],
    },
}

best_clf_dt = grid_search_pipeline(models_with_params, normalizer_params, vectorizers_with_params, X_train, y_train)

Grid Search Pipeline:
------------------

 mod: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
 vect: CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


Best parameter set is: {'clf__criterion': 'gini', 'clf__max_depth': 11, 'clf__min_samples_split': 30,

### Evaluation And Prediction:

In [28]:
text_clf_dt = best_clf_dt.best_estimator_
average_f1 = round(best_clf_dt.best_score_, 3)

pipe_dt = text_clf_dt.fit(X_train, y_train)
y_pred = text_clf_dt.predict(X_test)

print('Average f1: ', average_f1)
top_and_bottom_df(df_test, y_pred)

Average f1:  0.648


,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",m
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,m
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",m
322,322,היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...,m


--------------

## Trying Other Models

In [29]:
# svm -> LinearSVC
# linear_model -> Perceptron

models_with_params = {
    LinearSVC(): {'clf__loss': ['hinge', 'squared_hinge'],
                  'clf__dual': [True, False],
                  'clf__C': [0.01, 0.1, 1, 10, 50]
                 },
    Perceptron(): {'clf__alpha': [0.0001, 0.01],
                   'clf__penalty': [None, 'l2', 'l1', 'elasticnet']
                  },
}

normalizer_params = {'norm__norm': ['l2']}

vectorizers_with_params = {
    TfidfVectorizer(): {
        'vect__ngram_range': [(1,1)], 'vect__min_df': [1, 2],
        'vect__max_df': [1., 0.9], 'vect__sublinear_tf': [True, False],
    },
    CountVectorizer(): {
        'vect__ngram_range': [(1,1)], 'vect__min_df': [1, 2],
        'vect__max_df': [1., 0.9],
    },
}

best_clf = grid_search_pipeline(models_with_params, normalizer_params, vectorizers_with_params, X_train, y_train)

Grid Search Pipeline:
------------------

 mod: LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)
 vect: TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


Best parameter set is: {'clf__C': 50, 'clf__dual': True, 'clf__loss': 'hinge', 'norm__norm': 'l2', 'vect__max_df': 0.9, 'vect__min_df': 2, 'vect__ngram_range': (1, 1), 'vect__sublinear_tf': True} 

 And its score was: 

### Perceptron & Tfidf

Best parameter set was: {'clf__alpha': 0.0001, 'clf__penalty': 'l2', 'norm__norm': 'l2', 'vect__max_df': 0.9, 'vect__min_df': 2, 'vect__ngram_range': (1, 1), 'vect__sublinear_tf': True} 

And its score was: 0.704

----------------

## Trying more parameters on the most successful combination

### Perceptron & Tfidf

In [30]:
models_with_params = {
    Perceptron(): {'clf__alpha': [0.0001, 0.01, 0.000001],
                   'clf__penalty': [None, 'l2', 'l1', 'elasticnet']
                  },
}

normalizer_params = {'norm__norm': ['l2']}

vectorizers_with_params = {
    TfidfVectorizer(): {
        'vect__ngram_range': [(1,1), (2,2), (1,2)], 'vect__min_df': [2, 0.1],
        'vect__max_df': [0.9, 0.8], 'vect__sublinear_tf': [True, False],
    },

}

best_clf_perceptron = grid_search_pipeline(models_with_params, normalizer_params, vectorizers_with_params, X_train, y_train)

Grid Search Pipeline:
------------------

 mod: Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)
 vect: TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


Best parameter set is: {'clf__alpha': 1e-06, 'clf__penalty': 'l1', 'norm__norm': 'l2', 'vect__max_df': 0.8, 'vect__min_df': 2, 'vect__ngram_range': (1, 2), 'vect_

-------------

## The Final Model

### Perceptron & Tfidf:

Best parameter set is: {'clf__alpha': 1e-06, 'clf__penalty': 'l1',

'norm__norm': 'l2',

'vect__max_df': 0.8, 'vect__min_df': 2, 'vect__ngram_range': (1, 2), 'vect__sublinear_tf': True} 

In [32]:
text_clf_perceptron = best_clf_perceptron.best_estimator_

average_f1 = round(best_clf_perceptron.best_score_, 3)

### Evaluation And Prediction:

In [33]:
pipe_perceptron = text_clf_perceptron.fit(X_train, y_train)
y_pred = text_clf_perceptron.predict(X_test)

print('Average f1: ', average_f1)
top_and_bottom_df(df_test, y_pred)

Average f1:  0.726


,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",m
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,m
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",m
322,322,היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...,m


## Creates Predicted Data Frame:

In [34]:
df_predicted = get_df_predicted(df_test, y_pred)
df_predicted

,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
...,...,...,...
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",m
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,m
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",m


In [35]:
df_predicted.drop("story", axis=1, inplace=True)
df_predicted

,test_example_id,gender
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m


In [36]:
df_predicted['gender'].value_counts()

m    252
f     71
Name: gender, dtype: int64

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [37]:
df_predicted.to_csv('classification_results.csv',index=False)